In [8]:


# --- 1. Library Imports ---
# It's a best practice to group imports for clarity.

# Third-party libraries for data manipulation and analysis
import pandas as pd # type: ignore
import plotly.express as px  # type: ignore # Good to have visualization library here

# Third-party libraries for RDF and Knowledge Graph creation
from rdflib import Graph, Literal, Namespace, URIRef # type: ignore
from rdflib.namespace import RDF, RDFS, XSD # type: ignore

# Python standard library for URL handling
import urllib.parse
import re # Imported to create a more robust slugify function

# --- 2. Constants and Namespace Definitions ---
# Defining namespaces here keeps the code clean and easy to manage.
# Replace 'example.org' with a more descriptive base URI.
BASE_NS = Namespace("http://workplace-safety.example.org/ontology#")
RESOURCE_NS = Namespace("http://workplace-safety.example.org/resource/")

# --- 3. Utility Functions ---
# A well-documented utility function is a sign of a professional developer.

def slugify(text_to_slugify: str) -> str:
    """
    Cleans and prepares a string to be used safely in a URL or as a resource name.

    This function removes special characters, replaces spaces with underscores,
    and converts the string to lowercase. It's more robust than simple replace() calls.

    Args:
        text_to_slugify: The input string to be cleaned.

    Returns:
        A URL-safe "slug" version of the input string.
    """
    # Ensure the input is a string to avoid errors
    if not isinstance(text_to_slugify, str):
        text_to_slugify = str(text_to_slugify)
    
    # Remove special characters using regular expressions
    text_to_slugify = re.sub(r'[^\w\s-]', '', text_to_slugify)
    
    # Replace spaces and hyphens with a single underscore, and convert to lowercase
    return re.sub(r'[-\s]+', '_', text_to_slugify).lower()


def create_uri(namespace: Namespace, resource_name: str) -> URIRef:
    """
    Creates a full, URL-encoded URI for an RDF resource.

    Args:
        namespace: The RDFLib Namespace to use (e.g., for classes or resources).
        resource_name: The name of the resource (e.g., "Catania", "Construction_Industry").

    Returns:
        A valid URIRef object for the resource.
    """
    # First, create a clean, readable "slug" from the resource name
    slug = slugify(resource_name)
    
    # Then, URL-encode the slug to ensure it's a valid URI component
    # This is safer than relying on a custom replacement function.
    encoded_slug = urllib.parse.quote(slug)
    
    return namespace[encoded_slug]


In [9]:



import os # Imported to handle file paths robustly

# --- 1. Setup Project Paths ---
# This approach ensures the code runs on any computer without modification.
# The 'data' folder should be in the same directory as the Jupyter Notebook.
DATA_FOLDER = 'data'

# --- 2. Load the Main Workplace Incidents Dataset ---
# Renamed variables to English for clarity and international standards.
# Added error handling to provide a clear message if a file is missing.
try:
    incidents_path = os.path.join(DATA_FOLDER, 'DatiConCadenzaSemestraleInfortuniSicilia.csv')
    workplace_incidents = pd.read_csv(incidents_path, sep=";")
except FileNotFoundError:
    print(f"Error: The file {incidents_path} was not found. Make sure it's inside the '{DATA_FOLDER}' folder.")

# --- 3. Load Typological and Enrichment Datasets ---
# These are "lookup tables" used to add meaning to the main dataset.
# Using descriptive names makes the code's purpose much clearer.

# Province lookup table
provinces = pd.read_csv(os.path.join(DATA_FOLDER, 'Provincia.csv'), sep=";")

# Nation Codes lookup table (loaded directly from Excel)
# This avoids creating a temporary .csv file on your local machine, which is a major improvement.
nation_codes = pd.read_excel(os.path.join(DATA_FOLDER, 'CodiciNazioni.xls'))

# Administrative Definitions lookup table
admin_definitions = pd.read_csv(os.path.join(DATA_FOLDER, 'DefinizioneAmministrativa.csv'), sep=";")

# Compensation Type lookup table
compensation_types = pd.read_csv(os.path.join(DATA_FOLDER, 'TipologiaIndennizzo.csv'), sep=";")

# Fatal Outcome Decisions lookup table
# Specified 'latin-1' encoding as in the original code.
fatal_outcomes = pd.read_csv(os.path.join(DATA_FOLDER, 'DecisioneIstruttoriaEsitoMortale.csv'), sep=";", encoding='latin-1')

# Birthplace lookup table (assuming this is also a lookup table)
birthplaces = pd.read_csv(os.path.join(DATA_FOLDER, 'LuogoNascita.csv'), sep=";")

# --- 4. Initial Data Preview (Good Practice) ---
# It's always a good idea to display the first few rows of the main dataframe
# to confirm it has been loaded correctly.
print("Workplace incidents data loaded successfully. First 5 rows:")
print(workplace_incidents.head())

Workplace incidents data loaded successfully. First 5 rows:
  DataRilevazione DataProtocollo DataAccadimento DataDefinizione DataMorte  \
0      31/10/2022     18/11/2020      18/11/2020      25/11/2020       NaN   
1      31/10/2022     05/06/2019      24/05/2019      29/11/2019       NaN   
2      31/10/2022     20/07/2021      16/07/2021      01/11/2021       NaN   
3      31/10/2022     25/11/2019      20/11/2019      15/12/2019       NaN   
4      31/10/2022     12/05/2017      05/05/2017      20/02/2019       NaN   

   LuogoAccadimento  IdentificativoInfortunato Genere  Eta LuogoNascita  ...  \
0                88                   33031784      M   22         Z343  ...   
1                87                   18901323      M   55         ITAL  ...   
2                86                   28216801      M   53         ITAL  ...   
3                83                    5731062      M   58         ITAL  ...   
4                87                   11404309      M   62         ITAL

In [10]:
def clean_incidents_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans the raw workplace incidents DataFrame by removing unnecessary columns.

    This function takes the raw incidents DataFrame and drops a predefined list of
    columns that are not relevant for the analysis (e.g., identifiers, internal
    protocol dates).

    Args:
        df: The raw pandas DataFrame of workplace incidents.

    Returns:
        A new, cleaned pandas DataFrame with the specified columns removed.
    """
    # Define all columns to be dropped in a single, easy-to-read list.
    # This makes the code much more maintainable.
    columns_to_drop = [
        'DataRilevazione',
        'DataProtocollo',
        'DataDefinizione',
        'IdentificativoInfortunato',
        'GradoMenomazione',
        'IdentificativoDatoreLavoro',
        'PosizioneAssicurativaTerritoriale',
        'Gestione',
        'GestioneTariffaria',
        'GrandeGruppoTariffario',
        'DefinizioneAmministrativaEsitoMortale',
        'SettoreAttivitaEconomica',
        'ModalitaAccadimento'
    ]
    
    # Drop all columns in a single, efficient operation.
    # The .copy() is used to ensure the original DataFrame is not modified.
    cleaned_df = df.drop(columns=columns_to_drop).copy()
    
    # The redundant pd.concat line has been removed.
    
    print("DataFrame cleaned successfully. Columns removed.")
    return cleaned_df

# Call the function with the new, professional naming convention
cleaned_workplace_incidents = clean_incidents_dataframe(workplace_incidents)

DataFrame cleaned successfully. Columns removed.


In [11]:
def process_and_sort_by_date(df: pd.DataFrame, date_column: str, sort_keys: list) -> pd.DataFrame:
    """
    Processes date columns, extracts features, and sorts the DataFrame.

    This function converts the specified date column to a proper datetime format,
    creates a new 'year' column for easy analysis, and then sorts the DataFrame
    based on the provided keys. This approach is non-destructive, preserving
    the original full date information.

    Args:
        df: The input pandas DataFrame.
        date_column: The name of the column containing date strings.
        sort_keys: A list of column names to sort the DataFrame by.

    Returns:
        A new, processed, and sorted pandas DataFrame.
    """
    # It's a best practice to work on a copy to avoid unexpected side effects
    # on the original DataFrame (this prevents the SettingWithCopyWarning).
    processed_df = df.copy()
    
    # --- 1. Convert Date Column to Datetime Objects (The Right Way) ---
    # We convert the column but keep all the rich date information.
    processed_df[date_column] = pd.to_datetime(processed_df[date_column], format='%d/%m/%Y')

    # --- 2. Feature Engineering: Extract the Year into a NEW Column ---
    # This is the key improvement. We preserve the original datetime column
    # and create a new, separate column just for the year.
    processed_df['year'] = processed_df[date_column].dt.year
    
    # Now you could easily add more features if needed:
    # processed_df['month'] = processed_df[date_column].dt.month
    # processed_df['weekday'] = processed_df[date_column].dt.day_name()

    # --- 3. Sort the DataFrame ---
    # We use the full list of sort keys provided.
    processed_df = processed_df.sort_values(by=sort_keys)
    
    print(f"Date processing complete. '{date_column}' converted, 'year' column added, and DataFrame sorted.")
    return processed_df

# --- How to use the function ---
# Define the columns to sort by. Note that we can now sort by the new 'year' column.
# Let's assume you have an English column name 'location' instead of 'LuogoAccadimento'
# and the date column is 'incident_date'.
# For now, I will use your original names.
sort_order = ['LuogoAccadimento', 'DataAccadimento'] 
sorted_incidents = process_and_sort_by_date(
    df=cleaned_workplace_incidents, 
    date_column='DataAccadimento', 
    sort_keys=sort_order
)

# Display the result to confirm
# print(sorted_incidents[['DataAccadimento', 'year', 'LuogoAccadimento']].head())

Date processing complete. 'DataAccadimento' converted, 'year' column added, and DataFrame sorted.


In [12]:
def clean_and_filter_provinces(df: pd.DataFrame, region_to_keep: str) -> pd.DataFrame:
    """
    Cleans and filters the provinces DataFrame.

    This function removes unnecessary administrative columns and then filters the
    DataFrame to keep only the rows corresponding to a specific region.

    Args:
        df: The raw pandas DataFrame of provinces.
        region_to_keep: The name of the region to keep in the final dataset (e.g., "Sicilia").

    Returns:
        A new, cleaned, and filtered pandas DataFrame.
    """
    # Define all columns to be dropped in a single list for clarity.
    columns_to_drop = [
        "CodCittaMetropolitana", "CodRegione", "CodMacroregione", 
        "DescrMacroregione", "CodNazione", "DescrNazione", 
        "DataInizioValidita", "DataFineValidita"
    ]
    
    # --- The "Pandas-Idiomatic" Way: Chaining Operations ---
    # We first drop the columns, then apply the filter directly.
    # This is much more readable and efficient.
    
    # Using Boolean Indexing (most common method)
    filtered_df = df.drop(columns=columns_to_drop)
    # This line reads like English: "keep rows where DescrRegione is Sicilia"
    filtered_df = filtered_df[filtered_df['DescrRegione'] == region_to_keep].copy()

    # Alternative using .query() method (often even more readable)
    # cleaned_df = (
    #     df.drop(columns=columns_to_drop)
    #       .query(f"DescrRegione == '{region_to_keep}'")
    #       .copy()
    # )
    
    print(f"Provinces DataFrame cleaned and filtered for region: '{region_to_keep}'.")
    return filtered_df

# --- How to use the function ---
# We use the 'provinces' DataFrame we loaded earlier
sicilian_provinces = clean_and_filter_provinces(df=provinces, region_to_keep="Sicilia")

# print(sicilian_provinces.head())

Provinces DataFrame cleaned and filtered for region: 'Sicilia'.


In [13]:
def enrich_incidents_data(main_df: pd.DataFrame, lookup_dfs: dict) -> pd.DataFrame:
    """
    Enriches the main incidents DataFrame by merging it with multiple lookup tables.
    ... (il resto della docstring va bene) ...
    """
    enriched_df = main_df.copy()
    
    for original_col, (lookup_df, lookup_key, new_val_col) in lookup_dfs.items():
        # Perform the merge
        enriched_df = pd.merge(
            enriched_df,
            lookup_df,
            left_on=original_col,
            right_on=lookup_key,
            how='left'
        )
        
        # Drop the original code column and the now-redundant lookup key
        # CORREZIONE: Aggiunto un controllo per non eliminare la colonna due volte se i nomi coincidono
        columns_to_drop = {original_col, lookup_key}
        enriched_df = enriched_df.drop(columns=list(columns_to_drop))
        
        # Rename the new descriptive column to the original column's name for consistency
        enriched_df = enriched_df.rename(columns={new_val_col: original_col})

    return enriched_df


# --- How to use the function ---

# Il dizionario 'merge_plan' è corretto
merge_plan = {
    'LuogoAccadimento': (sicilian_provinces, 'Provincia', 'DescrProvincia'),
    'DefinizioneAmministrativa': (admin_definitions, 'DefinizioneAmministrativa', 'DescrDefinizioneAmministrativa'),
    'Indennizzo': (compensation_types, 'TipologiaIndennizzo', 'DescrTipologiaIndennizzo'),
    'DecisioneIstruttoriaEsitoMortale': (fatal_outcomes, 'DecisioneIstruttoriaEsitoMortale', 'DescrDecisioneIstruttoriaEsitoMortale'),
    'LuogoNascita': (birthplaces, 'LuogoNascita', 'DescrNazioneNascita')
}

# --- ECCO LA CORREZIONE ---
# Ho usato il nome corretto della funzione ('enrich_incidents_data')
# e il nome corretto dell'argomento ('lookup_dfs')
final_incidents_df = enrich_incidents_data(
    main_df=sorted_incidents, 
    lookup_dfs=merge_plan
)

# Ora puoi visualizzare l'anteprima del DataFrame corretto
print("DataFrame arricchito con successo. Ecco un'anteprima:")
print(final_incidents_df.head())

DataFrame arricchito con successo. Ecco un'anteprima:
  DataAccadimento DataMorte Genere  Eta ConSenzaMezzoTrasporto  \
0      2017-01-01       NaN      M   22                      N   
1      2017-01-01       NaN      F   64                      N   
2      2017-01-02       NaN      M   37                      N   
3      2017-01-02       NaN      F   55                      N   
4      2017-01-02       NaN      M   34                      N   

   IdentificativoCaso  GiorniIndennizzati  year CodSiglaProvincia  \
0            20385143                  48  2017                TP   
1            20403929                  15  2017                TP   
2            20423242                   0  2017                TP   
3            20438063                   1  2017                TP   
4            20388488                   0  2017                TP   

  LuogoAccadimento DescrRegione DefinizioneAmministrativa     Indennizzo  \
0          Trapani      Sicilia                  Positivo 

In [14]:
def handle_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Applies a strategic approach to handle missing values (NaNs).

    This function handles missing values differently for numeric and object columns:
    - Numeric columns: NaNs are filled with the median of the column.
    - Object (categorical) columns: NaNs are filled with the string 'Unknown'.
    This preserves data types and is a much more robust strategy.

    Args:
        df: The pandas DataFrame with missing values.

    Returns:
        A new DataFrame with NaNs handled strategically.
    """
    handled_df = df.copy()
    
    # Iterate over each column in the DataFrame
    for column in handled_df.columns:
        # Check if the column has any missing values
        if handled_df[column].isnull().any():
            # STRATEGY FOR NUMERIC COLUMNS
            if pd.api.types.is_numeric_dtype(handled_df[column]):
                # Fill with the median, which is robust to outliers
                median_value = handled_df[column].median()
                handled_df[column] = handled_df[column].fillna(median_value)
                print(f"Filled NaNs in numeric column '{column}' with median value ({median_value}).")
                
            # STRATEGY FOR CATEGORICAL/OBJECT COLUMNS
            elif pd.api.types.is_object_dtype(handled_df[column]):
                # Fill with a specific placeholder string
                handled_df[column] = handled_df[column].fillna('Unknown')
                print(f"Filled NaNs in object column '{column}' with 'Unknown'.")

    # After handling NaNs, it's a good practice to reset the index.
    handled_df = handled_df.reset_index(drop=True)
    
    print("\nMissing value handling and index reset complete.")
    return handled_df

# --- How to use the function ---
# Assuming 'final_incidents_df' is the output of the previous (merge) step
final_incidents_df_handled = handle_missing_values(final_incidents_df)

# Now, you can display the final, clean DataFrame
# final_incidents_df_handled

Filled NaNs in object column 'DataMorte' with 'Unknown'.
Filled NaNs in object column 'DataIngressoUE' with 'Unknown'.
Filled NaNs in object column 'DataUscitaUE' with 'Unknown'.

Missing value handling and index reset complete.


In [15]:
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, XSD, OWL # Import OWL for advanced concepts

def build_workplace_safety_ontology() -> Graph:
    """
    Builds the RDFS ontology for modeling workplace incidents.

    This ontology defines the core classes (Incident, Person, Location) and the
    properties that connect them, following professional semantic web conventions.

    Returns:
        An rdflib.Graph object containing the complete ontology.
    """
    g = Graph()

    # --- 1. Namespace Definitions (Professional Standard) ---
    INLO = Namespace("http://example.com/workplace-safety/ontology#")
    
    g.bind("inlo", INLO)
    g.bind("rdfs", RDFS)
    g.bind("owl", OWL)

    # --- 2. Class Definitions ---
    
    # The Person Class
    g.add((INLO.Person, RDF.type, RDFS.Class))
    g.add((INLO.Person, RDFS.label, Literal("Person", lang="en"))) # CORRETTO
    g.add((INLO.Person, RDFS.label, Literal("Persona", lang="it"))) # CORRETTO
    g.add((INLO.Person, RDFS.comment, Literal("Represents an individual involved in an incident.", lang="en"))) # CORRETTO

    # The Location Class
    g.add((INLO.Location, RDF.type, RDFS.Class))
    g.add((INLO.Location, RDFS.label, Literal("Location", lang="en"))) # CORRETTO
    g.add((INLO.Location, RDFS.label, Literal("Luogo", lang="it"))) # CORRETTO
    g.add((INLO.Location, RDFS.comment, Literal("A geographical location, such as a province.", lang="en"))) # CORRETTO
    
    # The Incident Class
    g.add((INLO.Incident, RDF.type, RDFS.Class))
    g.add((INLO.Incident, RDFS.label, Literal("Workplace Incident", lang="en"))) # CORRETTO
    g.add((INLO.Incident, RDFS.label, Literal("Incidente sul Lavoro", lang="it"))) # CORRETTO
    g.add((INLO.Incident, RDFS.comment, Literal("Represents a single workplace incident event.", lang="en"))) # CORRETTO

    # --- 3. Property Definitions ---
    
    # -- Person Properties --
    g.add((INLO.hasGender, RDF.type, RDF.Property))
    g.add((INLO.hasGender, RDFS.domain, INLO.Person))
    g.add((INLO.hasGender, RDFS.range, XSD.string))
    g.add((INLO.hasGender, RDFS.label, Literal("has gender", lang="en"))) # CORRETTO

    g.add((INLO.hasAge, RDF.type, RDF.Property))
    g.add((INLO.hasAge, RDFS.domain, INLO.Person))
    g.add((INLO.hasAge, RDFS.range, XSD.integer))
    g.add((INLO.hasAge, RDFS.label, Literal("has age", lang="en"))) # CORRETTO

    # -- Incident Properties --
    g.add((INLO.incidentDate, RDF.type, RDF.Property))
    g.add((INLO.incidentDate, RDFS.domain, INLO.Incident))
    g.add((INLO.incidentDate, RDFS.range, XSD.date))
    g.add((INLO.incidentDate, RDFS.label, Literal("incident date", lang="en"))) # CORRETTO
    
    g.add((INLO.daysIndemnified, RDF.type, RDF.Property))
    g.add((INLO.daysIndemnified, RDFS.domain, INLO.Incident))
    g.add((INLO.daysIndemnified, RDFS.range, XSD.integer))
    g.add((INLO.daysIndemnified, RDFS.label, Literal("days indemnified", lang="en"))) # CORRETTO

    # Object Properties (link classes to other classes)
    g.add((INLO.involvesPerson, RDF.type, RDF.Property))
    g.add((INLO.involvesPerson, RDFS.domain, INLO.Incident))
    g.add((INLO.involvesPerson, RDFS.range, INLO.Person))
    g.add((INLO.involvesPerson, RDFS.label, Literal("involves person", lang="en"))) # CORRETTO
    
    g.add((INLO.occurredAt, RDF.type, RDF.Property))
    g.add((INLO.occurredAt, RDFS.domain, INLO.Incident))
    g.add((INLO.occurredAt, RDFS.range, INLO.Location))
    g.add((INLO.occurredAt, RDFS.label, Literal("occurred at", lang="en"))) # CORRETTO

    return g

# --- How to use the function ---
if __name__ == "__main__":
    ontology_graph = build_workplace_safety_ontology()
    
    file_path = "workplace_safety_ontology.ttl"
    ontology_graph.serialize(destination=file_path, format='turtle')
    
    print(f"Ontology built and saved successfully to '{file_path}'.")
    print(f"Graph has {len(ontology_graph)} triples.")

Ontology built and saved successfully to 'workplace_safety_ontology.ttl'.
Graph has 36 triples.


In [ ]:
from tqdm.auto import tqdm # A library to create smart progress bars

def populate_graph_from_dataframe(df: pd.DataFrame, ontology_graph: Graph) -> Graph:
    """
    Populates the knowledge graph with instances from the DataFrame.

    This function iterates through the DataFrame efficiently using itertuples()
    and populates the graph according to the defined ontology.

    Args:
        df: The cleaned and enriched pandas DataFrame of incidents.
        ontology_graph: The graph containing the ontology definitions.

    Returns:
        The populated RDF graph.
    """
    # Work on a copy of the graph to avoid modifying the original ontology object
    populated_g = ontology_graph

    # Define namespaces (we use the ones from the ontology)
    INLO = Namespace("http://example.com/workplace-safety/ontology#")
    RESOURCE = Namespace("http://example.com/workplace-safety/resource/")
    DBR = Namespace("http://dbpedia.org/resource/")

    # Bind prefixes for cleaner output
    populated_g.bind("inlo", INLO)
    populated_g.bind("resource", RESOURCE)
    populated_g.bind("dbr", DBR)

    # Use df.itertuples() for a highly efficient iteration
    # tqdm adds a helpful progress bar for long operations
    print("Populating graph from DataFrame rows...")
    for row in tqdm(df.itertuples(), total=len(df)):
        # --- 1. Create URIs for the main instances ---
        # We use the robust create_uri function defined earlier
        incident_uri = create_uri(RESOURCE, f"incident_{row.IdentificativoCaso}")
        person_uri = create_uri(RESOURCE, f"person_{row.Index}") # Use the row index for a unique person ID
        location_uri = create_uri(RESOURCE, row.LuogoAccadimento)

        # --- 2. Add Triples for the Incident ---
        populated_g.add((incident_uri, RDF.type, INLO.Incident))
        populated_g.add((incident_uri, RDFS.label, Literal(f"Incident involving person {row.Index}", lang="en")))
        
        # Add data properties, checking for valid data before adding
        if pd.notna(row.DataAccadimento):
            # Assuming DataAccadimento is already a datetime object from previous steps
            # We format it to the correct XSD date format
            date_literal = Literal(row.DataAccadimento.date().isoformat(), datatype=XSD.date)
            populated_g.add((incident_uri, INLO.incidentDate, date_literal))

        if pd.notna(row.GiorniIndennizzati):
            populated_g.add((incident_uri, INLO.daysIndemnified, Literal(int(row.GiorniIndennizzati), datatype=XSD.integer)))

        # --- 3. Add Triples for the Person ---
        populated_g.add((person_uri, RDF.type, INLO.Person))
        if pd.notna(row.Genere):
             populated_g.add((person_uri, INLO.hasGender, Literal(row.Genere, datatype=XSD.string)))
        if pd.notna(row.Eta):
            populated_g.add((person_uri, INLO.hasAge, Literal(int(row.Eta), datatype=XSD.integer)))

        # --- 4. Add Triples for the Location ---
        populated_g.add((location_uri, RDF.type, INLO.Location))
        populated_g.add((location_uri, RDFS.label, Literal(row.LuogoAccadimento, lang="it")))
        
        # --- 5. Link Instances Together (Object Properties) ---
        populated_g.add((incident_uri, INLO.involvesPerson, person_uri))
        populated_g.add((incident_uri, INLO.occurredAt, location_uri))
        
        # --- 6. Link to External Data (DBpedia) - The Scalable Way ---
        # Dynamically create the DBpedia URI from the location name
        dbpedia_location_uri = create_uri(DBR, row.LuogoAccadimento)
        populated_g.add((location_uri, OWL.sameAs, dbpedia_location_uri))

    print("Graph population complete.")
    return populated_g

# --- How to use the function ---
if __name__ == "__main__":
    # 1. First, build the empty ontology structure
    ontology = build_workplace_safety_ontology()
    
    # 2. Then, populate it using our cleaned DataFrame
    # Note: Use the final, fully cleaned DataFrame from the previous steps
    final_graph = populate_graph_from_dataframe(
        df=final_incidents_df_handled, 
        ontology_graph=ontology
    )
    
    # 3. Serialize the final graph to a relative path
    output_path = "knowledge_graph.ttl"
    final_graph.serialize(destination=output_path, format='turtle')
    
    print(f"\nKnowledge Graph successfully created and saved to '{output_path}'.")
    print(f"Final graph has {len(final_graph)} triples.")


Populating graph from DataFrame rows...


c:\Users\Utente\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 131748/131748 [00:57<00:00, 2277.62it/s]


Graph population complete.

Knowledge Graph successfully created and saved to 'knowledge_graph.ttl'.
Final graph has 1185795 triples.


In [23]:

knowledge_graph = populate_graph_from_dataframe(
    df=final_incidents_df, 
    ontology_graph=ontology
)

output_filename = "workplace_safety_knowledge_graph.ttl"
knowledge_graph.serialize(destination=output_filename, format='turtle')

print(f"\nKnowledge Graph creato e salvato con successo in '{output_filename}'.")
print(f"Il grafo finale contiene {len(knowledge_graph)} triple.")



Populating graph from DataFrame rows...


100%|██████████| 131748/131748 [00:43<00:00, 3017.65it/s]


Graph population complete.

Knowledge Graph creato e salvato con successo in 'workplace_safety_knowledge_graph.ttl'.
Il grafo finale contiene 1185795 triple.


In [24]:
# The import for rdflib should be at the top of the script.

def execute_sparql_query(graph: Graph, query_string: str):
    """
    Executes a SPARQL query on a local rdflib graph and prints the results.

    Args:
        graph: The rdflib.Graph object to query.
        query_string: The SPARQL query to be executed.
    """
    print("--- Executing Query ---")
    print(query_string.strip())
    print("-----------------------")
    
    # We use the graph's native .query() method, which is simple and efficient.
    results = graph.query(query_string)
    
    if not results:
        print("Query returned no results.")
        return

    # The query result is an iterable object, so we can loop through it.
    print(f"Query found {len(results)} results. Showing up to 10:")
    for i, row in enumerate(results):
        # row.asdict() conveniently converts the result into a dictionary.
        print(row.asdict())
        if i == 9: # Limit the printout for readability.
            print(f"... (and {len(results) - 10} more)")
            break
            
    print("-----------------------\n")


# --- Example of How to Use the Function ---
# We use the 'knowledge_graph' object, which must already be created and populated in a previous cell.
# There is NO NEED to save and re-parse the file from disk.

# --- Example Query 1: Find all incidents that occurred in Catania ---
query_incidents_in_catania = """
    PREFIX inlo: <http://example.com/workplace-safety/ontology#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?incident ?date
    WHERE {
        ?incident a inlo:Incident ;
                  inlo:occurredAt ?location ;
                  inlo:incidentDate ?date .
        
        ?location rdfs:label "Catania"@it .
    }
    ORDER BY ?date
    LIMIT 10
"""
execute_sparql_query(knowledge_graph, query_incidents_in_catania)


# --- Example Query 2: Count the number of incidents per province ---
query_count_by_province = """
    PREFIX inlo: <http://example.com/workplace-safety/ontology#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?province (COUNT(?incident) AS ?incidentCount)
    WHERE {
        ?incident a inlo:Incident ;
                  inlo:occurredAt ?location .
        
        ?location rdfs:label ?province .
    }
    GROUP BY ?province
    ORDER BY DESC(?incidentCount)
"""
execute_sparql_query(knowledge_graph, query_count_by_province)

--- Executing Query ---
PREFIX inlo: <http://example.com/workplace-safety/ontology#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?incident ?date
    WHERE {
        ?incident a inlo:Incident ;
                  inlo:occurredAt ?location ;
                  inlo:incidentDate ?date .

        ?location rdfs:label "Catania"@it .
    }
    ORDER BY ?date
    LIMIT 10
-----------------------
Query found 10 results. Showing up to 10:
{'incident': rdflib.term.URIRef('http://example.com/workplace-safety/resource/incident_20401491'), 'date': rdflib.term.Literal('2017-01-01', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date'))}
{'incident': rdflib.term.URIRef('http://example.com/workplace-safety/resource/incident_20428506'), 'date': rdflib.term.Literal('2017-01-01', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date'))}
{'incident': rdflib.term.URIRef('http://example.com/workplace-safety/resource/incident_20431809'), 'date': rdflib.term.

In [25]:
# --- Query 1: Find all people born in Italy ---
# This query demonstrates a robust, case-insensitive filter.

query_people_born_in_italy = """
    PREFIX inlo: <http://example.com/workplace-safety/ontology#>

    SELECT ?person ?birthPlace
    WHERE {
        ?person a inlo:Person ;
                inlo:hasBirthPlace ?birthPlace .
        
        # This FILTER is more robust:
        # 1. It converts the birthplace to lowercase before comparing.
        # 2. It checks if the birthplace string contains "italia".
        FILTER(CONTAINS(LCASE(?birthPlace), "italia"))
    }
    LIMIT 10
"""

# --- Execute the query using our standard function ---
# We use the 'knowledge_graph' object that is already in memory.
execute_sparql_query(knowledge_graph, query_people_born_in_italy)

--- Executing Query ---
PREFIX inlo: <http://example.com/workplace-safety/ontology#>

    SELECT ?person ?birthPlace
    WHERE {
        ?person a inlo:Person ;
                inlo:hasBirthPlace ?birthPlace .

        # This FILTER is more robust:
        # 1. It converts the birthplace to lowercase before comparing.
        # 2. It checks if the birthplace string contains "italia".
        FILTER(CONTAINS(LCASE(?birthPlace), "italia"))
    }
    LIMIT 10
-----------------------
Query returned no results.


In [26]:
# --- Query 2: Find all incidents that occurred in Palermo ---
# This query shows how to traverse relationships in the graph.

query_incidents_in_palermo = """
    PREFIX inlo: <http://example.com/workplace-safety/ontology#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?incident ?date
    WHERE {
        # 1. First, find the entity (?location) that is a Location and has the Italian label "Palermo".
        ?location a inlo:Location ;
                  rdfs:label "Palermo"@it .
        
        # 2. Then, find any ?incident that is linked to that ?location
        #    via the 'occurredAt' property.
        ?incident a inlo:Incident ;
                  inlo:occurredAt ?location ;
                  inlo:incidentDate ?date .
    }
    ORDER BY ?date
    LIMIT 10
"""

# --- Execute the query using our standard function ---
execute_sparql_query(knowledge_graph, query_incidents_in_palermo)

--- Executing Query ---
PREFIX inlo: <http://example.com/workplace-safety/ontology#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?incident ?date
    WHERE {
        # 1. First, find the entity (?location) that is a Location and has the Italian label "Palermo".
        ?location a inlo:Location ;
                  rdfs:label "Palermo"@it .

        # 2. Then, find any ?incident that is linked to that ?location
        #    via the 'occurredAt' property.
        ?incident a inlo:Incident ;
                  inlo:occurredAt ?location ;
                  inlo:incidentDate ?date .
    }
    ORDER BY ?date
    LIMIT 10
-----------------------
Query found 10 results. Showing up to 10:
{'incident': rdflib.term.URIRef('http://example.com/workplace-safety/resource/incident_20371223'), 'date': rdflib.term.Literal('2017-01-01', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date'))}
{'incident': rdflib.term.URIRef('http://example.com/workplace-safety/resourc

In [27]:
# --- Query 3: Count incidents per province and order them ---
# This query demonstrates aggregation (GROUP BY) to find the provinces
# with the highest number of incidents.

query_count_by_province = """
    PREFIX inlo: <http://example.com/workplace-safety/ontology#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?provinceLabel (COUNT(?incident) AS ?incidentCount)
    WHERE {
        # 1. Find all incidents.
        ?incident a inlo:Incident .
        
        # 2. Follow the 'occurredAt' link from the incident to its location.
        ?incident inlo:occurredAt ?location .
        
        # 3. Get the Italian label from that location.
        ?location rdfs:label ?provinceLabel .
    }
    GROUP BY ?provinceLabel
    ORDER BY DESC(?incidentCount)
"""

# --- Execute the query using our standard function ---
execute_sparql_query(knowledge_graph, query_count_by_province)

--- Executing Query ---
PREFIX inlo: <http://example.com/workplace-safety/ontology#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?provinceLabel (COUNT(?incident) AS ?incidentCount)
    WHERE {
        # 1. Find all incidents.
        ?incident a inlo:Incident .

        # 2. Follow the 'occurredAt' link from the incident to its location.
        ?incident inlo:occurredAt ?location .

        # 3. Get the Italian label from that location.
        ?location rdfs:label ?provinceLabel .
    }
    GROUP BY ?provinceLabel
    ORDER BY DESC(?incidentCount)
-----------------------
Query found 9 results. Showing up to 10:
{'provinceLabel': rdflib.term.Literal('Catania', lang='it'), 'incidentCount': rdflib.term.Literal('33021', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer'))}
{'provinceLabel': rdflib.term.Literal('Palermo', lang='it'), 'incidentCount': rdflib.term.Literal('29016', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#inte

In [28]:
# --- Query 4: Count incidents per year to identify trends ---
# This query demonstrates how to use SPARQL functions (like YEAR())
# to perform more advanced temporal analysis.

query_count_by_year = """
    PREFIX inlo: <http://example.com/workplace-safety/ontology#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT ?year (COUNT(?incident) AS ?incidentCount)
    WHERE {
        # 1. Find all incidents that have an incidentDate property.
        ?incident a inlo:Incident ;
                  inlo:incidentDate ?date .
        
        # 2. Use the YEAR() function to extract the year part from the date literal.
        BIND(YEAR(?date) AS ?year)
    }
    GROUP BY ?year
    ORDER BY ?year
"""

# --- Execute the query using our standard function ---
execute_sparql_query(knowledge_graph, query_count_by_year)

--- Executing Query ---
PREFIX inlo: <http://example.com/workplace-safety/ontology#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT ?year (COUNT(?incident) AS ?incidentCount)
    WHERE {
        # 1. Find all incidents that have an incidentDate property.
        ?incident a inlo:Incident ;
                  inlo:incidentDate ?date .

        # 2. Use the YEAR() function to extract the year part from the date literal.
        BIND(YEAR(?date) AS ?year)
    }
    GROUP BY ?year
    ORDER BY ?year
-----------------------
Query found 5 results. Showing up to 10:
{'year': rdflib.term.Literal('2017', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), 'incidentCount': rdflib.term.Literal('28763', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer'))}
{'year': rdflib.term.Literal('2018', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), 'incidentCount': rdflib.term.Literal('28251', datatype=rdflib.term.URIRef('h

In [29]:
# The import for plotly.express should be at the top of the script
# with the other imports.

def generate_visualizations(df: pd.DataFrame):
    """
    Generates and displays key visualizations from the final DataFrame.

    Args:
        df: The final, cleaned, and enriched pandas DataFrame.
    """
    print("\n--- Generating Visualizations ---")

    # It's a best practice to first prepare the data for the plot.
    # We'll use English column names for the plots.
    # Let's assume the final df has columns: 'LuogoAccadimento', 'Genere'
    # We can rename them for plotting if needed, or use them directly if already renamed.
    
    # --- Visualization 1: Bar Chart of Incidents per Province ---
    # A bar chart is better than a histogram for categorical data.
    # Sorting the values makes the chart much more insightful.
    
    province_counts = df['LuogoAccadimento'].value_counts().reset_index()
    province_counts.columns = ['Province', 'Incident Count']
    
    fig_bar = px.bar(
        province_counts.sort_values('Incident Count', ascending=False),
        x='Province',
        y='Incident Count',
        title="Total Workplace Incidents by Province (2017-2021)",
        labels={'Province': 'Province in Sicily', 'Incident Count': 'Total Number of Incidents'},
        text_auto=True,      # Displays the count on top of each bar
        template='plotly_white' # A clean, professional theme
    )
    fig_bar.update_traces(marker_color='royalblue') # A more professional color
    fig_bar.show()

    # --- Visualization 2: Bar Chart for Gender Distribution ---
    # While a pie chart works, bar charts are often easier to interpret accurately.
    gender_counts = df['Genere'].value_counts().reset_index()
    gender_counts.columns = ['Gender', 'Count']
    
    fig_pie = px.pie(
        gender_counts, 
        names='Gender', 
        values='Count',
        title="Distribution of Incidents by Gender",
        hole=0.3, # A donut chart is often more visually appealing
        template='plotly_white'
    )
    fig_pie.update_traces(textinfo='percent+label', marker_colors=['skyblue', 'salmon'])
    fig_pie.show()

# --- Example of How to Use the Function ---
# Call this function at the very end of your script,
# after the DataFrame is fully cleaned and processed.
generate_visualizations(final_df)

NameError: name 'final_df' is not defined

In [ ]:
# Prima crea una cartella 'images' nella directory del tuo progetto!
fig_bar.write_image("images/incidents_by_province.png")
fig_pie.write_image("images/incidents_by_gender.png")